In [1]:
import tensorflow as tf

/home/y2536zha/anaconda3/envs/watermark/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/y2536zha/anaconda3/envs/watermark/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/y2536zha/anaconda3/envs/watermark/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/y2536zha/anaconda3/envs/w

In [2]:
from tensorflow import keras

from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from IPython.core.interactiveshell import InteractiveShell
from time import time
from copy import deepcopy
import pandas as pd

import matplotlib as plt

InteractiveShell.ast_node_interactivity = "all"
pd.options.display.max_columns = None
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_rows', 100)

%matplotlib inline 
import sys, os
base_path = os.getcwd()[0:os.getcwd().rfind('Watermark')] + "Watermark/"
sys.path.append(base_path) 

import matplotlib.pyplot as plt
from src.asiaccs_main import asiaccs_blackbox
from src.models import get_deep_cnn_for_cifar, get_lenet_model_for_mnist
from src.preprocess_data import load_cifar_images, load_mnist_images, transform_to_one_hot
from src.util import plot_blackbox

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Using TensorFlow backend.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [3]:
import numpy as np

In [4]:
train, test = load_cifar_images()

In [6]:
x_train, y_train = train
x_test, y_test = test

x_train_original = x_train[:25000]
x_train_surrogate = x_train[25000:]

y_train_original = y_train[:25000]
y_train_surrogate = y_train[25000:]

del x_train
del y_train

x_train_original.shape
y_train_original.shape

x_train_surrogate.shape
y_train_surrogate.shape

x_test.shape
y_test.shape

(25000, 32, 32, 3)

(25000, 10)

(25000, 32, 32, 3)

(25000, 10)

(10000, 32, 32, 3)

(10000, 10)

In [7]:
model_original = get_deep_cnn_for_cifar() 

In [8]:
model_original.load_weights('model/best_cifar_model.h5') # original model is trained on x_train_original only

In [9]:
model_original.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)       

In [10]:
from tensorflow.keras import backend as K
from tensorflow.keras.layers import multiply, Input

In [11]:
def compute_jacob(x_train, y_train, model):
    y_label_byhost = Input((10,))
    output_masked = multiply([model.output, y_label_byhost])
    
    jcob = tf.gradients(ys = [output_masked],
                        xs = [model.input],)[0]
    
    tmp_fun = K.function([model.input, y_label_byhost],
                     [jcob])
    
    return tmp_fun([x_train, y_train])[0]
    

In [12]:
class TerminateOnBaseline(keras.callbacks.Callback):
    """Callback that terminates training when either acc or val_acc reaches a specified baseline
    """
    def __init__(self, monitor='acc', baseline=0.9):
        super(TerminateOnBaseline, self).__init__()
        self.monitor = monitor
        self.baseline = baseline

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        acc = logs.get(self.monitor)
        if acc is not None:
            if acc >= self.baseline:
                print('Epoch %d: Reached baseline, terminating training' % (epoch))
                self.model.stop_training = True

In [13]:
x_train_attack = x_train_surrogate[:150]

In [14]:
y_train_attack = model_original.predict(x_train_attack).argmax(axis=1)
y_train_attack = transform_to_one_hot(y_train_attack, depth=10)

In [15]:
y_train_attack.shape
y_train_attack.shape
x_test.shape
y_test.shape

(150, 10)

(150, 10)

(10000, 32, 32, 3)

(10000, 10)

In [21]:
def Practical_black_box_attach(model_surrogate_fun, # function used to build the surrogate model
                               x_train_attack,
                               y_train_attack,
                               x_test,
                               y_test,
                               lambda_update = 1, # lamda used, as seen in the original paper
                               num_iter = 10,
                               eval_metric = 'acc',
                               delta = 0.1,
                               path_store_surrogate_model = 'model/best_mnist_model_surrogate_Black.h5',
                               batch_size_train = 64,
                               batch_size_eval = 1024,
                               epochs = 10):
    
    # Check Basic setting
    if eval_metric not in model_original.metrics_names:
        raise Exception('Error: eval_metric not in the original model')
        
    eval_metric_index = model_original.metrics_names.index(eval_metric)
    model_original_performance = model_original.evaluate(x_test, y_test, batch_size = batch_size_eval, verbose = 0)[eval_metric_index]
    
    print('Now show basic setting')
    print('original model performance:', eval_metric, model_original_performance)
    print('use', eval_metric, 'as the evaluation metric,', 'delta = ', delta,
      ',stealing will stop as long as the validation', eval_metric, 'achieve', model_original_performance - delta)
    
    
    # Preparation for training
    print('Now prepare for training')
    callbacks = TerminateOnBaseline(monitor = 'val_' + eval_metric, baseline = model_original_performance - delta)
    checkpoint_surrogate = keras.callbacks.ModelCheckpoint( path_store_surrogate_model, verbose=1, monitor='val_acc',save_best_only=True, mode='auto') 
    model_surrogate = model_surrogate_fun()
    
    # train model 
    history_list = []
    for i in range(num_iter):
        print('iteration:', i, 'x_train_attack shape:', x_train_attack.shape, 'y_train_attack shape:', y_train_attack.shape)
        history = model_surrogate.fit(x_train_attack,
                                      y_train_attack,
                                      batch_size = batch_size_train, 
                                      epochs = epochs, 
                                      verbose = 1,
                                      validation_data = (x_test, y_test),
                                      callbacks = [checkpoint_surrogate, callbacks])
        history_list.append(history)
        
        model_surrogate_performance = model_surrogate.evaluate(x_test, y_test, batch_size = batch_size_eval, verbose = 0)[eval_metric_index]
        if model_surrogate_performance >= model_original_performance - delta:
            print('Reached the desired performance, break the loop')
            break;

        # update synthetic dataset
        jacob = np.sign(compute_jacob(x_train_attack, y_train_attack, model_surrogate)) 
        x_train_attack_added = x_train_attack + lambda_update * jacob
        y_train_attack_added = model_original.predict(x_train_attack_added).argmax(axis=1)
        y_train_attack_added = transform_to_one_hot(y_train_attack_added, depth=10)

        x_train_attack = np.concatenate([x_train_attack, x_train_attack_added])
        y_train_attack = np.concatenate([y_train_attack, y_train_attack_added])
        
    return (model_surrogate, history_list)

In [22]:
import gc
gc.collect()

20

In [ ]:
model_surrogate, history_list = Practical_black_box_attach(model_surrogate_fun = get_deep_cnn_for_cifar,
                               x_train_attack = x_train_attack,
                               y_train_attack = y_train_attack,
                               x_test = x_test,
                               y_test = y_test,
                               lambda_update = 1,
                               num_iter = 10,
                               eval_metric = 'acc',
                               delta = 0.3,
                               path_store_surrogate_model = 'model/best_cifar_model_surrogate_Black.h5',
                               batch_size_train = 64,
                               batch_size_eval = 1024,
                               epochs = 10)

Now show basic setting
original model performance: acc 0.7595
use acc as the evaluation metric, delta =  0.3 ,stealing will stop as long as the validation acc achieve 0.45950002670288087
Now prepare for training
iteration: 0 x_train_attack shape: (150, 32, 32, 3) y_train_attack shape: (150, 10)
Train on 150 samples, validate on 10000 samples
Epoch 1/10
128/150 [========================>.....] - ETA: 0s - loss: 3.9547 - acc: 0.1406
Epoch 00001: val_acc improved from -inf to 0.10140, saving model to model/best_cifar_model_surrogate_Black.h5
150/150 [==============================] - 22s 150ms/sample - loss: 3.8901 - acc: 0.1267 - val_loss: 2.3319 - val_acc: 0.1014
Epoch 2/10
128/150 [========================>.....] - ETA: 0s - loss: 1.6663 - acc: 0.5156
Epoch 00002: val_acc improved from 0.10140 to 0.14060, saving model to model/best_cifar_model_surrogate_Black.h5
150/150 [==============================] - 16s 105ms/sample - loss: 1.6788 - acc: 0.5200 - val_loss: 2.3021 - val_acc: 0.1406

576/600 [===========================>..] - ETA: 0s - loss: 0.1096 - acc: 0.9757
Epoch 00008: val_acc improved from 0.19300 to 0.21000, saving model to model/best_cifar_model_surrogate_Black.h5
600/600 [==============================] - 19s 32ms/sample - loss: 0.1110 - acc: 0.9750 - val_loss: 4.7330 - val_acc: 0.2100
Epoch 9/10
576/600 [===========================>..] - ETA: 0s - loss: 0.2795 - acc: 0.9271
Epoch 00009: val_acc did not improve from 0.21000
600/600 [==============================] - 19s 31ms/sample - loss: 0.2745 - acc: 0.9283 - val_loss: 6.4363 - val_acc: 0.1701
Epoch 10/10
576/600 [===========================>..] - ETA: 0s - loss: 0.2860 - acc: 0.9288
Epoch 00010: val_acc did not improve from 0.21000
600/600 [==============================] - 19s 32ms/sample - loss: 0.2820 - acc: 0.9300 - val_loss: 6.2829 - val_acc: 0.1243
iteration: 3 x_train_attack shape: (1200, 32, 32, 3) y_train_attack shape: (1200, 10)
Train on 1200 samples, validate on 10000 samples
Epoch 1/10
115

Epoch 6/10
4736/4800 [============================>.] - ETA: 0s - loss: 0.0941 - acc: 0.9774
Epoch 00006: val_acc did not improve from 0.26370
4800/4800 [==============================] - 46s 10ms/sample - loss: 0.0930 - acc: 0.9777 - val_loss: 6.2713 - val_acc: 0.2350
Epoch 7/10
4736/4800 [============================>.] - ETA: 0s - loss: 0.0667 - acc: 0.9814
Epoch 00007: val_acc did not improve from 0.26370
4800/4800 [==============================] - 50s 10ms/sample - loss: 0.0665 - acc: 0.9815 - val_loss: 7.8672 - val_acc: 0.2458
Epoch 8/10
4736/4800 [============================>.] - ETA: 0s - loss: 0.0604 - acc: 0.9833
Epoch 00008: val_acc did not improve from 0.26370
4800/4800 [==============================] - 52s 11ms/sample - loss: 0.0608 - acc: 0.9833 - val_loss: 6.5080 - val_acc: 0.2307
Epoch 9/10
4736/4800 [============================>.] - ETA: 0s - loss: 0.0512 - acc: 0.9854
Epoch 00009: val_acc did not improve from 0.26370
4800/4800 [==============================] - 53

Epoch 6/10
38336/38400 [============================>.] - ETA: 0s - loss: 0.0089 - acc: 0.9972
Epoch 00006: val_acc did not improve from 0.27400
38400/38400 [==============================] - 228s 6ms/sample - loss: 0.0091 - acc: 0.9972 - val_loss: 21.5079 - val_acc: 0.1873
Epoch 7/10
38336/38400 [============================>.] - ETA: 0s - loss: 0.0115 - acc: 0.9972
Epoch 00007: val_acc did not improve from 0.27400
38400/38400 [==============================] - 233s 6ms/sample - loss: 0.0116 - acc: 0.9971 - val_loss: 22.5687 - val_acc: 0.1818
Epoch 8/10
38336/38400 [============================>.] - ETA: 0s - loss: 0.0099 - acc: 0.9972
Epoch 00008: val_acc did not improve from 0.27400
38400/38400 [==============================] - 225s 6ms/sample - loss: 0.0099 - acc: 0.9972 - val_loss: 12.0967 - val_acc: 0.2123
Epoch 9/10
38336/38400 [============================>.] - ETA: 0s - loss: 0.0088 - acc: 0.9976
Epoch 00009: val_acc did not improve from 0.27400
38400/38400 [=================